In [2]:
# pip install requests bs4 tqdm

In [3]:
import requests
import os
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin, urlparse

In [11]:
import matplotlib.pylab as plt

In [17]:
import functools
import tensorflow as tf

In [4]:
def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [45]:
url = "https://www.volvocars.com/zh-cn/v/cars/c40-electric"

In [46]:
parsed = urlparse(url)
print(parsed.netloc)
print(parsed.scheme)

www.volvocars.com
https


In [47]:
is_valid(url)

True

In [8]:
def get_all_images(url):
    """
    Returns all image URLs on a single `url`
    """
    soup = bs(requests.get(url).content, "html.parser")
    
    urls = []
    
    for img in tqdm(soup.find_all("img"), "Extracting images"):
        img_url = img.attrs.get("src")
        if not img_url:
            # if img does not contain src attribute, just skip
            continue
        
        # make the URL absolute by joining domain with the URL that is just extracted
        img_url = urljoin(url, img_url)
        try:
            pos = img_url.index("?")
            img_url = img_url[:pos]
        except ValueError:
            pass
        # finally, if the url is valid
        if is_valid(img_url):
            urls.append(img_url)
    return urls

In [48]:
imgs = get_all_images(url)

Extracting images: 100%|██████████| 18/18 [00:00<00:00, 37957.50it/s]


In [49]:
imgs = [x for x in imgs if '.jpg' in x]

In [50]:
imgs

['https://www.volvocars.com/images/v/cn/-/media/market-assets/china/applications/dotcom/images/pdp/my22/c40/gallery-grid/c40-gallary-grid/c40-recharge-scattered-gallery-1-4x3.jpg',
 'https://www.volvocars.com/images/v/cn/-/media/project/contentplatform/data/media/my22/c40/c40-recharge-scattered-gallery-3-4x3.jpg',
 'https://www.volvocars.com/images/v/cn/-/media/project/contentplatform/data/media/my22/c40/c40-recharge-two-tiles-grid-2-1x1.jpg',
 'https://www.volvocars.com/images/v/cn/-/media/market-assets/china/applications/dotcom/images/my22/hero-image/c40-recharge-promoted-vehicle-4x5.jpg']

In [56]:
def download(url, pathname):
    """
    Downloads a file given an URL and puts it in the folder `pathname`
    """
    # if path doesn't exist, make that path dir
    if not os.path.isdir(pathname):
        os.makedirs(pathname)
    # download the body of response by chunk, not immediately
    response = requests.get(url, stream=True)
    # get the total file size
    file_size = int(response.headers.get("Content-Length", 0))
    # get the file name
    filename = os.path.join(pathname, url.split("/")[-1])
    # progress bar, changing the unit to bytes instead of iteration (default by tqdm)
    progress = tqdm(response.iter_content(1024), f"Downloading {filename}", total=file_size, unit="B", unit_scale=True, unit_divisor=1024)
    with open(filename, "wb") as f:
        for data in progress.iterable:
            # write data read to the file
            f.write(data)
            # update the progress bar manually
            progress.update(len(data))

In [57]:
path = 'img/'
for img in imgs:
    # for each image, download it
    download(img, path)

In [18]:
def crop_center(image):
    """Returns a cropped square image."""
    shape = image.shape
    new_shape = min(shape[1], shape[2])
    offset_y = max(shape[1] - shape[2], 0) // 2
    offset_x = max(shape[2] - shape[1], 0) // 2
    image = tf.image.crop_to_bounding_box(image, offset_y, offset_x, new_shape, new_shape)
    return image

@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), preserve_aspect_ratio=True):
    """Loads and preprocesses images."""
    # Cache image file locally.
    image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
    # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
    img = tf.io.decode_image(
        tf.io.read_file(image_path),
        channels=3, dtype=tf.float32)[tf.newaxis, ...]
    img = crop_center(img)
    img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
    return img

In [51]:
content_urls = dict()

for i in imgs:
    t = i.split('/')[-1].split('.')[0]
    content_urls[t] = i

content_urls

{'c40-recharge-scattered-gallery-1-4x3': 'https://www.volvocars.com/images/v/cn/-/media/market-assets/china/applications/dotcom/images/pdp/my22/c40/gallery-grid/c40-gallary-grid/c40-recharge-scattered-gallery-1-4x3.jpg',
 'c40-recharge-scattered-gallery-3-4x3': 'https://www.volvocars.com/images/v/cn/-/media/project/contentplatform/data/media/my22/c40/c40-recharge-scattered-gallery-3-4x3.jpg',
 'c40-recharge-two-tiles-grid-2-1x1': 'https://www.volvocars.com/images/v/cn/-/media/project/contentplatform/data/media/my22/c40/c40-recharge-two-tiles-grid-2-1x1.jpg',
 'c40-recharge-promoted-vehicle-4x5': 'https://www.volvocars.com/images/v/cn/-/media/market-assets/china/applications/dotcom/images/my22/hero-image/c40-recharge-promoted-vehicle-4x5.jpg'}

In [55]:
# content_image_size = 384

# content_images = {k: load_image(v, (content_image_size, content_image_size)) for k, v in content_urls.items()}